In [3]:
import re  #正規化
import math
import datetime as dt
import time 
import requests
from bs4 import BeautifulSoup as BS
import json
import ast  #轉換成json需要套件
startdate = dt.datetime(2016,3,1)
endate = dt.datetime(2016,3,7)
totaldays = (endate - startdate).days + 1 #期間有幾天
count=0
k=0
all_json=''
for daynumber in range(totaldays):
    datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y-%m-%d") 
    print datestring
    #每一天的蘋果新聞頁面
    #http://www.chinatimes.com/history-by-date/2016-03-19-2601?page=1 
    page_url = 'http://www.chinatimes.com/history-by-date/{}-2601?page=1'.format(datestring)
    #print page_url
    res_test=requests.get(page_url)
    soup_test=BS(res_test.text)
    total_pages1=soup_test.select('.listLeft ul')[0].select('li')[0].select('span')[1]
    #print total_pages1
    total_pages2=(str(total_pages1.text))
    lis=soup_test.select('.listRight li')
    how_many_new_in_page=len(lis)
    dic=[total_pages2,"apple"]
    for ele in dic: 
        #print dic
        m=re.search('\((?P<username>\d+)\)',ele)
        if m:
            total_pages=int(m.group('username'))   #總頁數
            #print total_pages
        
    how_many_page=int(math.ceil(total_pages/float(how_many_new_in_page))) #有多少分頁
    #print how_many_page
    
    #  以上得知 ↑↑↑   ------→ 每一天的新聞，各自有多少篇數，還有多少分頁 ---------------------------
    
    for j in xrange(how_many_page+1):
        secondPages='http://www.chinatimes.com/history-by-date/{}-2601?page={}'.format(datestring,j)  #僅此2016-03-19-2601，之後要做不同天的!
        print secondPages  
        time.sleep(3)    #要睡喔! 不然的話會失敗，網頁沒開完整 就開始抓就會容易失敗!!!!!!!!!!!!!!!!!! Q_Q
    #  以上得知 ↑↑↑   ------→ 每一天的新聞不同的分頁數 ---------------------------
        url=secondPages
        res=requests.get(url)
        soup=BS(res.text)
        
        #print soup
        mainPage='http://www.chinatimes.com/'
        lis=soup.select('.listRight li')
        how_many_new_in_page=len(lis)
        l=0
        #print how_many_new_in_page
        
        for li in lis:
            print l
            l=l+1
            if l<12:   #單頁報導要寫入幾天 
                title=li.select('h2')[0].text.strip() #list→unicode  加了[0]→'bs4.element.Tag' 可顯示中文字
                #print title  #2016/3/23 ok
                category=li.select('.kindOf a')[0].text.strip() #類別
                #print category # 2016/3/23 ok
                Data2=soup.select('.page_index li')[1]
                Data1=Data2.select('h6 a')[0]['href'][0:].split('-')
                Data=Data1[0]+Data1[1]+Data1[2]              #yyyymmdd  日期    
                #print Data   # 2016/3/23 ok
                newspaper_office='中國時報'                  #報社
                page1=li.select('h2 a')[0]['href'][0:]      #新聞連結，進入到內文的階段
                each_new_page_url=mainPage+page1 
                #print each_new_page_url  # 2016/3/23 ok
                second_url=requests.get(each_new_page_url)
                second_usoup=BS(second_url.text)  
                hits=second_usoup.findAll('div',{'class':'art_click clear-fix'})[0]
                number_of_hits = int(hits.select('span')[1].text)            #點擊率
                all_p_tags=second_usoup.select('.clear-fix p')[:-6]
                content_list=[]
                for p_tag in all_p_tags:
                    p_tag1=str(p_tag).replace("，","_")
                    p_tag2=str(p_tag1).replace("。","_")
                    p_tag3=str(p_tag2).replace("<p>","")
                    p_tag4=str(p_tag3).replace("</p>","")
                    #print p_tag4    #2016/3/23 ok                         #內文(已將 "，" & "。"轉換)
                    content_list.append(p_tag4)                            #將全部 標籤 P寫入 list中
                separator=''
                joined_content=separator.join(content_list)              #將 list [i] 以 "" 合併，得到全文內容    |獨家秘技!!!!| 
                #print joined_content  #2016/3/23 ok
                keyword=second_usoup.select('.a_k a')
                i=0
                kwords_list=[]
                for kwords in range(len(keyword)):
                    keyword=second_usoup.select('.a_k a')[i]
                    #print keyword.text                                       #此頁新聞的關鍵字
                    i=i+1
                    kwords_list.append(keyword.text)
                separator='_'    
                joined_kwords=separator.join(kwords_list)               #將 list [i] 以 "" 合併，得到關鍵字全部內容 
                #print joined_kwords    #2016/3/23 ok

                m = {
                     'url':each_new_page_url,'title':title,'category':category,\
                     'content':joined_content,'date':Data,'hitcount':number_of_hits,\
                     'keyw':joined_kwords,'comp':'ChinaTimes'
                     }                          #填入json格式dict

                all_json+=str(m)+','            #dict不可以先加 所以先變STR相加   
                time.sleep(0.3) 
            else:                           #把每一篇的新聞存成m
                break

#2016/3/23 ok #2016/0324  4:58 start  5:52  (2016-03-01-2016-03-05-2601?page=7) "IndexError: list index out of range"

2016-03-01
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=0
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=1
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=2
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=3
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=4
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=5
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=6
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=7
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=8
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=9
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatimes.com/history-by-date/2016-03-01-2601?page=10
0
1
2
3
4
5
6
7
8
9
10
http://www.chinatime

In [ ]:
#轉檔案

In [6]:
a=eval(all_json)                       #AST套件 轉換 all_json 成 json
d1 =  json.dumps(a) 
d =  json.loads(d1)
for z in range(0,len(d)):
    print d[z]['title'],d[z]['category'],d[z]['date'],d[z]['hitcount']  #查詢KEY

 小英：電價10年不大漲 焦點要聞 20160301 19435
救股市 陸央行降準2碼 焦點要聞 20160301 5804
好看好玩 228連假擠爆 台灣燈會人潮創紀錄 焦點要聞 20160301 3490
逛趟台灣燈會 賞盡東亞美景 焦點要聞 20160301 6809
乞龜、博餅、科技雲 好看更好玩 焦點要聞 20160301 696
在地融合 展現多元族群風情 焦點要聞 20160301 1813
370萬人次創紀錄 不髒不亂 賞燈撿垃圾 萬名志工：不辛苦 焦點要聞 20160301 2319
大眼蛙超人氣 呂文忠找回初衷 焦點要聞 20160301 1040
台達電創辦人鄭崇華：不漲電價？ 不要為難蔡英文 焦點要聞 20160301 2611
改善產學落差 蔡英文：教長人選 要能與產業溝通 焦點要聞 20160301 975
綠能穩10年電價 太陽能業者：有難度 台電：做好迎戰準備 焦點要聞 20160301 4356
小英：電價10年不大漲 焦點要聞 20160301 19435
救股市 陸央行降準2碼 焦點要聞 20160301 5804
好看好玩 228連假擠爆 台灣燈會人潮創紀錄 焦點要聞 20160301 3490
逛趟台灣燈會 賞盡東亞美景 焦點要聞 20160301 6809
乞龜、博餅、科技雲 好看更好玩 焦點要聞 20160301 696
在地融合 展現多元族群風情 焦點要聞 20160301 1813
370萬人次創紀錄 不髒不亂 賞燈撿垃圾 萬名志工：不辛苦 焦點要聞 20160301 2319
大眼蛙超人氣 呂文忠找回初衷 焦點要聞 20160301 1040
台達電創辦人鄭崇華：不漲電價？ 不要為難蔡英文 焦點要聞 20160301 2611
改善產學落差 蔡英文：教長人選 要能與產業溝通 焦點要聞 20160301 975
綠能穩10年電價 太陽能業者：有難度 台電：做好迎戰準備 焦點要聞 20160301 4356
3.2萬鉛管用戶 暫不調價 雙北152萬戶 今起水費漲3成 焦點要聞 20160301 3716
台水轄區廣大 暫不調漲 焦點要聞 20160301 497
鴻夏戀再拖 結連理恐成怨偶 焦點要聞 20160301 21815
夏普設新部門 專責與鴻海談判 焦點要聞 20160301 1185
小英主張高教機自製 

In [ ]:
#寫入json檔案

In [7]:
import json
with open('C:/Users/BIG DATA/pythonETL/project/news/20160301_20160307.json', 'w') as f:
    f.write(d1) 

In [ ]:
#讀出json檔案

In [ ]:
import json
with open('C:/Users/BIG DATA/pythonETL/project/news/20160301_20160305(P7-2).json', 'r') as f:
    d =  json.loads(d1)
    for z in range(0,len(d)):
        print d[z]['title'],d[z]['sort'],d[z]['time'],d[z]['people']  #查詢KEY